# **Multi-Agent System for Cloud Architecture Recommendation**

In [13]:
from typing import Dict, Any, List, Optional
import time
import pprint

In [7]:
class RequirementsAnalyst:
    """Analyse le problème métier et produit une spécification structurée."""
    def analyze(self, problem: str, context: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        ctx = context or {}
        spec = {
            "raw_description": problem,
            "functional": [],
            "non_functional": [],
            "expected_load": ctx.get("expected_load", {}),
            "privacy": ctx.get("privacy", "standard"),
            "integrations": ctx.get("integrations", [])
        }
        p = problem.lower()
        # heuristiques basiques (pour les scénarios fournis)
        if "e-commerce" in p or "store" in p or "shopping" in p:
            spec["functional"] = ["product_catalog", "cart", "checkout", "admin_dashboard", "user_auth"]
            spec["expected_load"] = ctx.get("expected_load", {"daily_users": 1000, "peak_rps": 20})
            spec["non_functional"] = ["high_availability", "scalability", "secure_payments"]
            if "payment" in p or "checkout" in p:
                spec["privacy"] = "pci"
        elif "chatbot" in p or "support" in p or "conversation" in p:
            spec["functional"] = ["nlp_chat", "session_store", "escalation_to_human", "crm_sync"]
            spec["expected_load"] = ctx.get("expected_load", {"conversations_per_day": 500, "concurrency": 10})
            spec["non_functional"] = ["low_latency", "observability"]
        elif "expense" in p or "expense" in p:
            spec["functional"] = ["receipt_upload", "ocr_processing", "approval_workflow", "payroll_integration"]
            spec["expected_load"] = ctx.get("expected_load", {"monthly_users": 500})
            spec["non_functional"] = ["data_retention", "privacy"]
            spec["privacy"] = "sensitive"
        else:
            # fallback: try to use provided context
            spec["functional"] = ctx.get("functional", [])
            spec["non_functional"] = ctx.get("non_functional", [])
        return spec

In [8]:
class ArchitectureDesigner:
    """Transforme la spec en proposition d'architecture (niveau conceptuel)."""
    def design(self, spec: Dict[str, Any]) -> Dict[str, Any]:
        comp = {}
        notes = []
        load = spec.get("expected_load", {})

        # Compute
        if spec["functional"] and ("nlp_chat" in spec["functional"] or "ocr_processing" in spec["functional"]):
            # heavy AI tasks -> mix of managed AI and containers
            comp["compute"] = {"api": "containers (managed)", "workers": "serverless or batch workers", "ai": "managed/hosted model or GPU cluster (if needed)"}
            notes.append("AI/ML workloads demand special compute; consider managed services")
        else:
            comp["compute"] = {"api": "containers (managed/fargate)", "workers": "serverless for light tasks"}

        # Database & Storage
        if "checkout" in spec["functional"] or spec["privacy"] == "pci":
            comp["database"] = {"type": "relational", "engine": "managed (RDS/Aurora/Postgres)", "reason": "transactions/orders"}
            notes.append("Use tokenized payments and do not store raw card data")
        elif "session_store" in spec["functional"] or "nlp_chat" in spec["functional"]:
            comp["database"] = {"type": "NoSQL", "engine": "managed (Dynamo/Firestore)", "reason": "session and chat state"}
        else:
            comp["database"] = {"type": "relational", "engine": "managed"}

        comp["object_storage"] = {"type": "object_storage", "reason": "images, receipts, static assets (S3/GCS/Azure Blob)"}
        comp["cache"] = {"type": "in-memory", "engine": "Redis/ElastiCache", "reason": "hot reads, sessions, performance"}

        # Networking
        comp["api_gateway"] = {"type": "API Gateway / Load Balancer", "reason": "routing, auth, rate limiting"}
        comp["cdn"] = {"type": "CDN (CloudFront/Cloud CDN)", "reason": "static assets and performance"} if ("product_catalog" in spec["functional"] or "images" in spec.get("integrations", [])) else {"type": "optional"}

        # Observability & Security
        comp["observability"] = {"metrics": ["latency", "error_rate", "throughput"], "logging": "centralized", "traces": True}
        comp["security"] = {"tls": True, "secrets": "use_secret_manager", "iam": "least_privilege", "waf": "recommended"}

        # Integrations
        comp["integrations"] = spec.get("integrations", [])
        if spec.get("privacy") == "pci":
            notes.append("PCI compliance: use external payment provider, separate network zones, strict logging & retention")

        # Cost hints
        est = {"cost_hint": "low" if (load.get("daily_users", 0) <= 1000 and not spec.get("nlp")) else "medium"}

        architecture = {"components": comp, "notes": notes, "estimated": est}
        return architecture

In [9]:
class SecurityAgent:
    """Vérifie les points de sécurité de base et signale les risques majeurs."""
    def review(self, architecture: Dict[str, Any], spec: Dict[str, Any]) -> Dict[str, Any]:
        issues = []
        comp_sec = architecture.get("components", {}).get("security", architecture.get("components", {}).get("security", {}))
        # simple checks
        if not architecture["components"].get("api_gateway"):
            issues.append("Missing API Gateway - affects auth and rate-limiting")
        if spec.get("privacy") == "pci":
            issues.append("Requires PCI compliance: avoid storing card data; use tokenization and validated gateway")
        # always recommend secret manager and least privilege
        recommendations = ["use_secret_manager", "apply_least_privilege_iam", "enable_tls", "enable_logging/auditing"]
        severity = "high" if issues else "low"
        return {"issues": issues, "recommendations": recommendations, "severity": severity}

In [10]:
class CostEvaluator:
    """Donne une estimation heuristique et propose des alternatives si besoin."""
    def evaluate(self, architecture: Dict[str, Any], spec: Dict[str, Any], budget: Optional[float] = None) -> Dict[str, Any]:
        # Heuristique simple basée sur presence de components
        score = 0
        comps = architecture.get("components", {})
        # base per-component cost
        if comps.get("compute"):
            score += 100
        if comps.get("database"):
            score += 150
        if comps.get("object_storage"):
            score += 10
        if comps.get("cache"):
            score += 30
        if comps.get("observability"):
            score += 20
        # scale with load
        load = spec.get("expected_load", {})
        if load.get("daily_users", 0) > 10000:
            multiplier = 3
        elif load.get("daily_users", 0) > 1000:
            multiplier = 1.5
        else:
            multiplier = 1.0
        estimated = round(score * multiplier, 2)
        within_budget = True if (budget is None or estimated <= budget) else False
        recommendations = []
        if not within_budget:
            recommendations.append("Consider serverless for burst tasks or reduce retention for logs / metrics")
        return {"estimated_monthly_usd": estimated, "within_budget": within_budget, "recommendations": recommendations}


class IntegrationAgent:
    """Propose comment intégrer les systèmes externes (CRM, payment, payroll...)."""
    def plan(self, spec: Dict[str, Any], architecture: Dict[str, Any]) -> Dict[str, Any]:
        plans = []
        for integ in spec.get("integrations", []):
            if "crm" in integ.lower():
                plans.append({"target": "CRM", "auth": "OAuth2 or API key", "adapter": "build connector service"})
            elif "payment" in integ.lower() or spec.get("privacy") == "pci":
                plans.append({"target": "PaymentGateway", "notes": "Use managed PCI-compliant gateway & tokenization"})
            elif "payroll" in integ.lower():
                plans.append({"target": "Payroll", "notes": "Push approved expenses via secure API"})
            else:
                plans.append({"target": integ, "notes": "Unknown integration - validate API & auth"})
        if not plans:
            plans.append({"target": "none", "notes": "No external integrations required or provided"})
        return {"integration_plan": plans}

In [11]:
class Orchestrator:
    def __init__(self):
        self.req = RequirementsAnalyst()
        self.designer = ArchitectureDesigner()
        self.cost = CostEvaluator()
        self.sec = SecurityAgent()
        self.intg = IntegrationAgent()

    def run(self, problem: str, context: Optional[Dict[str, Any]] = None, budget: Optional[float] = None, max_iters: int = 3) -> Dict[str, Any]:
        trace: List[Dict[str, Any]] = []
        spec = self.req.analyze(problem, context)
        arch = self.designer.design(spec)

        for i in range(1, max_iters + 1):
            cost_report = self.cost.evaluate(arch, spec, budget)
            sec_report = self.sec.review(arch, spec)
            int_report = self.intg.plan(spec, arch)

            step = {
                "iteration": i,
                "spec": spec,
                "architecture": arch,
                "cost": cost_report,
                "security": sec_report,
                "integration": int_report
            }
            trace.append(step)

            # simple convergence: accept if within budget and no critical security issues
            if cost_report["within_budget"] and sec_report["severity"] == "low":
                final = {"spec": spec, "architecture": arch, "cost": cost_report, "security": sec_report, "integration": int_report, "converged": True}
                return {"trace": trace, "final": final}
            # otherwise try small adjustments (cost-saving / security hardening)
            # cost-saving heuristic: convert some containers -> serverless if present
            comps = arch.get("components", {})
            modified = False
            for k, v in comps.items():
                if isinstance(v, dict) and v.get("type") == "containers (managed)":
                    v["type"] = "serverless (where possible)"
                    modified = True
                    break
            if not modified:
                # if no containers, enforce security recommendations
                arch.setdefault("notes", []).append("Applied security recommendations")
                arch.setdefault("components", {}).setdefault("security", {})["hardened"] = True

        # if not converged after iterations, return best-effort
        final = {"spec": spec, "architecture": arch, "cost": cost_report, "security": sec_report, "integration": int_report, "converged": False}
        return {"trace": trace, "final": final}

In [12]:
def pretty_print(result: Dict[str, Any]) -> None:
    pp = pprint.PrettyPrinter(indent=2, width=120)
    if result.get("final"):
        print("\n=== FINAL RECOMMENDATION ===\n")
        pp.pprint(result["final"])
    else:
        print("No final recommendation produced.")
    print("\n-- Trace summary (iterations):")
    for step in result.get("trace", []):
        print(f" Iteration {step['iteration']}: cost_est={step['cost']['estimated_monthly_usd']} within_budget={step['cost']['within_budget']} security_severity={step['security']['severity']}")


# Demo runs for the two sample scenarios
if __name__ == "__main__":
    orch = Orchestrator()

    # Scenario A: Simple E-commerce Site
    problem_a = "Simple E-commerce site with product catalog, cart, checkout and admin dashboard (payment processing)"
    result_a = orch.run(problem_a, context={"expected_load": {"daily_users": 1000}}, budget=500.0)
    pretty_print(result_a)

    # Scenario B: Customer Support Chatbot
    problem_b = "Customer Support Chatbot integrated with CRM, handle 500+ conversations per day and escalate to humans"
    result_b = orch.run(problem_b, context={"expected_load": {"conversations_per_day": 500}}, budget=300.0)
    pretty_print(result_b)


=== FINAL RECOMMENDATION ===

{ 'architecture': { 'components': { 'api_gateway': { 'reason': 'routing, auth, rate limiting',
                                                     'type': 'API Gateway / Load Balancer'},
                                    'cache': { 'engine': 'Redis/ElastiCache',
                                               'reason': 'hot reads, sessions, performance',
                                               'type': 'in-memory'},
                                    'cdn': { 'reason': 'static assets and performance',
                                             'type': 'CDN (CloudFront/Cloud CDN)'},
                                    'compute': { 'api': 'containers (managed/fargate)',
                                                 'workers': 'serverless for light tasks'},
                                    'database': { 'engine': 'managed (RDS/Aurora/Postgres)',
                                                  'reason': 'transactions/orders',
              